<a href="https://colab.research.google.com/github/anupshinde04/Vaccine-Receiver-Prediction-Model/blob/main/Anup_DataHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DataHack by IIT Guwahati and GeeksforGeeks
# **Vaccine Receiver Prediction Model**
By Anup Shinde

# Importing necessary Libraries

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [127]:
import xgboost as xgb
from sklearn.preprocessing import MultiLabelBinarizer

# Getting started with data

Importing the data

In [137]:
train_features = pd.read_csv('/content/drive/My Drive/training_set_features.csv')
train_labels = pd.read_csv('/content/drive/My Drive/training_set_labels.csv')
test_features = pd.read_csv('/content/drive/My Drive/test_set_features.csv')

Finding Basic Details

In [4]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)

(26707, 36)
(26707, 3)
(26708, 36)


In [6]:
train_features.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [7]:
train_labels.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [8]:
test_features.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


# Data Preprocessing

Merge the training features and labels together

In [138]:
train_data = train_features.merge(train_labels, on='respondent_id')

In [10]:
train_data.head()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,xyz_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [11]:
train_data.dtypes

respondent_id                    int64
xyz_concern                    float64
xyz_knowledge                  float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_xyz                float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_xyz_vacc_effective     float64
opinion_xyz_risk               float64
opinion_xyz_sick_from_vacc     float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                      

Finding Null Values

In [12]:
train_data.isnull().sum()

respondent_id                      0
xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

separating target columns

In [139]:
X = train_data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
Y = train_data[['xyz_vaccine', 'seasonal_vaccine']]
y_xyz = train_data['xyz_vaccine']
y_seasonal = train_data['seasonal_vaccine']

Handling missing data

In [140]:
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

X_en = preprocessor.fit_transform(X)

Splitting the data

In [47]:
X_train, X_val, y_train_xyz, y_val_xyz = train_test_split(X_en, y_xyz, test_size=0.3, random_state=73)
X_train, X_val, y_train_seasonal, y_val_seasonal = train_test_split(X_en, y_seasonal, test_size=0.3, random_state=73)

# Defining Models

**Logistic Regression**

Initializing the model

In [73]:
model_xyz = LogisticRegression(max_iter=3000)
model_seasonal = LogisticRegression(max_iter=3000)

In [74]:
model_xyz.fit(X_train, y_train_xyz)
model_seasonal.fit(X_train, y_train_seasonal)

LogisticRegression(max_iter=3000)

predicting probability

In [75]:
pred_proba_xyz_val = model_xyz.predict_proba(X_val)[:, 1]
pred_proba_seasonal_val = model_seasonal.predict_proba(X_val)[:, 1]

Calculating ROC AUC Score

In [76]:
auc_xyz = roc_auc_score(y_val_xyz, pred_proba_xyz_val)
auc_seasonal = roc_auc_score(y_val_seasonal, pred_proba_seasonal_val)
auc_average = ((auc_seasonal+auc_xyz)/2)

In [77]:
print(f'ROC AUC for xyz_vaccine: {auc_xyz:.4f}')
print(f'ROC AUC for seasonal_vaccine: {auc_seasonal:.4f}')
print(f'Average ROC AUC: {auc_average:.4f}')

ROC AUC for xyz_vaccine: 0.8326
ROC AUC for seasonal_vaccine: 0.8486
Average ROC AUC: 0.8406


Predictions on test features

In [53]:
X_test = test_features.drop(['respondent_id'], axis=1)
X_test_encoded = preprocessor.transform(X_test)

In [54]:
pred_proba_xyz_test = model_xyz.predict_proba(X_test_encoded)[:, 1]
pred_proba_seasonal_test = model_seasonal.predict_proba(X_test_encoded)[:, 1]

Saving Submission File

In [56]:
submission_df = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': pred_proba_xyz_test,
    'seasonal_vaccine': pred_proba_seasonal_test
})

In [57]:
submission_df.to_csv('/content/drive/My Drive/submission_format.csv', index=False)

print("Submission saved successfully.")

Submission saved successfully.


**Random Forest Model**

Training the model

In [104]:
model = MultiOutputClassifier(RandomForestClassifier(random_state=73))

In [105]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

In [106]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=71)

In [107]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_to...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation'],
      dtype='object'))])),
                ('model',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=73)))])

Predictions on test features

In [190]:
y_pred_proba = pipeline.predict_proba(X_val)
roc_auc_xyz = roc_auc_score(y_val['xyz_vaccine'], [prob[1] for prob in y_pred_proba[0]])
roc_auc_seasonal = roc_auc_score(y_val['seasonal_vaccine'], [prob[1] for prob in y_pred_proba[1]])

ValueError: X has 105 features, but ColumnTransformer is expecting 35 features as input.

In [109]:
print(f'ROC AUC for XYZ Vaccine: {roc_auc_xyz}')
print(f'ROC AUC for Seasonal Vaccine: {roc_auc_seasonal}')
print(f'Average ROC AUC: {(roc_auc_xyz + roc_auc_seasonal) / 2}')

ROC AUC for XYZ Vaccine: 0.8508233245924643
ROC AUC for Seasonal Vaccine: 0.854992163853081
Average ROC AUC: 0.8529077442227726


Saving Submission File

In [110]:
submission_df = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': pred_proba_xyz_test,
    'seasonal_vaccine': pred_proba_seasonal_test
})

In [111]:
submission_df.to_csv('/content/drive/My Drive/submission_format2.csv', index=False)

print("Submission saved successfully.")

Submission saved successfully.


**XGBoost Classification**

In [141]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 6,
    'eta': 0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'verbosity': 0
}

Converting object type column into boolean for this model

In [142]:
cat_cols = ['age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
train_data = pd.get_dummies(train_data, columns=cat_cols)
test_features = pd.get_dummies(test_features, columns=cat_cols)

In [143]:
mlb = MultiLabelBinarizer()
X = mlb.fit_transform(X_en)

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Training the model

In [186]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [187]:
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Finding probability

In [192]:
# Predict probabilities for the test set
y_prob = model.predict_proba(X_test)

# Compute ROC AUC score for each class
roc_auc = roc_auc_score(y_test, y_prob, average='macro')
print(f"ROC AUC Score: {roc_auc}")


ROC AUC Score: 0.844373318760348
